<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [ ]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [7]:
# Create SQL query using natality data after the year 2000
import google.cloud.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [15]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,5896567601480310696,342825
1,9068386407968572094,330863
2,260598435387740869,320332
3,2700219941840496344,307395
4,4740473290291881219,359717


hashmonth와 모듈러 연산을 사용하고, RAND로 그 중 랜덤하게 몇퍼센트만 추출하여 샘플링하자.

In [14]:
# MOD(ABS(hashmonth), 10) < 8, 80%의 데이터만 추출해서 그 갯수가 몇개인지 본다.
sampling_query = "SELECT COUNT(weight_pounds) FROM (" + query + ") WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0004"
print(sampling_query)

SELECT COUNT(weight_pounds) FROM (
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
) WHERE MOD(ABS(hashmonth), 10) < 8 AND RAND() < 0.0004


In [13]:
df = bq.Client().query(sampling_query).to_dataframe()
df

,f0_
0,10692


이제 실질적으로 데이터를 추출하여, train과 eval데이터로 나눈다.

In [19]:
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
train_df = bq.Client().query(trainQuery).to_dataframe()
eval_df = bq.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(train_df), len(eval_df)))

There are 13427 examples in the train dataset and 3332 in the eval dataset


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

먼저, train_df의 데이터 상세 정보에 대해 살펴보자. 아래 표를 보면 각 feature별 데이터 개수가 각각 13414개, 13427개... 등으로 다른 것을 알 수 있다. 그렇다면 feature별 데이터 개수가 모두 같도록 전처리 해줘야 한다.

In [21]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13414.000000,13427.000000,13427.000000,13333.000000,1.342700e+04
mean,7.241250,27.299918,1.033812,38.619815,4.401843e+18
std,1.328792,6.151709,0.191167,2.605440,2.795475e+18
min,0.500449,13.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,1.000000,38.000000,1.622638e+18
50%,7.330370,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.170970e+18
max,14.186747,53.000000,4.000000,47.000000,9.183606e+18


In [22]:
train_df.head(10)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.874912,True,21,1,40.0,774501970389208065
1,8.375361,True,26,1,40.0,774501970389208065
2,6.437498,False,20,1,38.0,774501970389208065
3,8.375361,True,20,1,41.0,774501970389208065
4,5.436599,True,23,1,40.0,774501970389208065
5,6.812284,True,19,1,37.0,774501970389208065
6,8.437091,True,27,1,39.0,774501970389208065
7,7.813183,True,23,1,41.0,774501970389208065
8,6.876218,True,37,1,38.0,774501970389208065
9,7.125340,True,27,1,38.0,774501970389208065


In [31]:
# train데이터와 eval데이터에 null개수
print("NULL values in train data:\n{}\nNULL values eval data:\n{}".format(train_df.isnull().sum(), eval_df.isnull().sum()))

NULL values in train data:
weight_pounds      13
is_male             0
mother_age          0
plurality           0
gestation_weeks    94
hashmonth           0
dtype: int64
NULL values eval data:
weight_pounds       1
is_male             0
mother_age          0
plurality           0
gestation_weeks    29
hashmonth           0
dtype: int64


In [39]:
# 전처리를 해주자!
import pandas as pd
def preprocess(df):
  # drop NULL values
  df.dropna(inplace=True)
  
  # modify plurality feature to be a string
  # {1: 'Single(1)', 2: 'Twins(2)', 3: 'Triplets(3)', 4: 'Quadruplets(4)', 5: 'Quintuplets(5)'}
  plur_dic = dict(zip([1,2,3,4,5],
                     ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  
  # Dictionary 이용, 1 -> Single(1)과 같이 바꿔주는 식
  df['plurality'].replace(plur_dic, inplace=True)
  
  # Now create extra rows to simulate lack of ultrasound(성별 및 몇쌍둥이 여부를 알 수 없게 한다.)
  df2 = df.copy(deep=True) # 별도의 DataFrame 생성
  df2.loc[df2['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  df2['is_male'] = 'Unknown'
  
  return pd.concat([df, df2])

In [40]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
# 전처리 함수 통과 후 바뀐 데이터를 체크하자.
train_df.head(10)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.874912,True,21,Single(1),40.0,774501970389208065
1,8.375361,True,26,Single(1),40.0,774501970389208065
2,6.437498,False,20,Single(1),38.0,774501970389208065
3,8.375361,True,20,Single(1),41.0,774501970389208065
4,5.436599,True,23,Single(1),40.0,774501970389208065
5,6.812284,True,19,Single(1),37.0,774501970389208065
6,8.437091,True,27,Single(1),39.0,774501970389208065
7,7.813183,True,23,Single(1),41.0,774501970389208065
8,6.876218,True,37,Single(1),38.0,774501970389208065
9,7.125340,True,27,Single(1),38.0,774501970389208065


In [41]:
train_df.tail(10)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13417,6.012006,Unknown,26,Single(1),38.0,6637442812569910270
13418,7.500126,Unknown,19,Single(1),42.0,6637442812569910270
13419,6.999677,Unknown,25,Single(1),38.0,6637442812569910270
13420,6.999677,Unknown,21,Single(1),42.0,6637442812569910270
13421,7.374463,Unknown,23,Single(1),39.0,6637442812569910270
13422,7.054792,Unknown,26,Single(1),41.0,6637442812569910270
13423,7.813183,Unknown,31,Single(1),40.0,6637442812569910270
13424,7.473671,Unknown,37,Single(1),39.0,6637442812569910270
13425,7.403123,Unknown,27,Single(1),39.0,6637442812569910270
13426,3.688334,Unknown,18,Multiple(2+),31.0,6637442812569910270


In [42]:
# .describe()는 오직 numerical value만 보여주므로, plurality는 보이지 않는다.
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,213168.000000,213168.000000,213168.000000,2.131680e+05
mean,7.243656,27.300983,38.625009,4.403823e+18
std,1.327130,6.149696,2.584269,2.794920e+18
min,0.500449,13.000000,18.000000,1.244589e+17
25%,6.563162,22.000000,38.000000,1.622638e+18
50%,7.341393,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.170970e+18
max,14.186747,53.000000,47.000000,9.183606e+18


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [43]:
train_df.to_csv('train.csv', index=False, header=False)
eval_df.to_csv('eval.csv', index=False, header=False)

In [44]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   52848 eval.csv
  213168 train.csv
  266016 total
==> eval.csv <==
8.375361333379999,True,28,Single(1),40.0,1088037545023002395
9.43798943622,True,19,Single(1),40.0,1088037545023002395
6.944561253,False,37,Single(1),38.0,74931465496927487
6.9996768185,True,30,Single(1),38.0,1891060869255459203
7.7492485093,True,27,Single(1),39.0,1891060869255459203
8.61786982158,True,31,Single(1),38.0,7146494315947640619
7.43839671988,False,30,Single(1),40.0,6782146986770280327
8.68841774542,True,21,Single(1),39.0,1639186255933990135
5.8753192823,False,24,Single(1),37.0,1891060869255459203
7.5618555866,False,44,Single(1),36.0,2246942437170405963

==> train.csv <==
7.87491199864,True,21,Single(1),40.0,774501970389208065
8.375361333379999,True,26,Single(1),40.0,774501970389208065
6.4374980503999994,False,20,Single(1),38.0,774501970389208065
8.375361333379999,True,20,Single(1),41.0,774501970389208065
5.43659938092,True,23,Single(1),40.0,774501970389208065
6.8122838958,True,19,Single(1),37.0,774501970389

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License